In [ ]:
from pysmt.shortcuts import Symbol, Equals, GT, And, Or, Solver, Int, Not, Minus, Times, Plus, Div
from pysmt.typing import INT


def declare(i):
    state = {}
    state['s'] = Symbol('s'+str(i),INT)
    state['t'] = Symbol('t'+str(i),INT)
    state['r'] = Symbol('r'+str(i),INT)
    state['s_prox'] = Symbol('s_prox'+str(i),INT)
    state['t_prox'] = Symbol('t_prox'+str(i),INT)
    state['r_prox'] = Symbol('r_prox'+str(i),INT)
    state['q'] = Symbol('q'+str(i),INT)
    state['pc'] = Symbol('pc'+str(i),INT)
    
    return state



def init(state, a, b):

    A = Equals(state['pc'], Int(0))
    B = GT(Int(a), Int(0))
    C = GT(Int(b), Int(0))
    D = Equals(state['r'], Int(a))
    E = Equals(state['s'], Int(1))
    F = Equals(state['t'], Int(0))
    
    return And(A, B, C, D, E, F)

def trans(curr, prox, a, b):
           
    t01 = And(
        Equals(curr['pc'], Int(0)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['r'], Int(a)),
        Equals(prox['r_prox'], Int(b)),
        Equals(prox['s'], Int(1)),
        Equals(prox['s_prox'], Int(0)),
        Equals(prox['t'], Int(0)),
        Equals(prox['t_prox'], Int(1)),
        Equals(curr['q'], prox['q'])
    )
    
    t14 = And(
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(4)),
        Equals(curr['r_prox'], Int(0)),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )

    t12 = And(
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(2)),
        Not(Equals(curr['r_prox'], Int(0))),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )
    
    division = Div(curr['r'], curr['r_prox'])
    
    t23 = And(
        Equals(curr['pc'], Int(2)),
        Equals(prox['pc'], Int(3)),
        Equals(prox['q'], division),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox'])
    )
    
    times_q_r_prox = Times(curr['q'], curr['r_prox'])
    times_q_s_prox = Times(curr['q'], curr['s_prox'])
    times_q_t_prox = Times(curr['q'], curr['t_prox'])
    
    minus_r_times = Minus(curr['r'], times_q_r_prox)
    minus_s_times = Minus(curr['s'], times_q_s_prox)
    minus_t_times = Minus(curr['t'], times_q_t_prox)

    t31 = And(
        Equals(curr['pc'], Int(3)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['r'], curr['r_prox']),
        Equals(prox['r_prox'], minus_r_times),
        Equals(prox['s'], curr['s_prox']),
        Equals(prox['s_prox'], minus_s_times),
        Equals(prox['t'], curr['t_prox']),
        Equals(prox['t_prox'], minus_t_times),
        Equals(prox['q'], curr['q'])
    )

    t44 = And(
        Equals(curr['pc'], Int(4)),
        Equals(prox['pc'], Int(4)),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )
    
    return Or(t01, t14, t12, t23, t31, t44)

def gera_traco(i, a, b, K):
    
    with Solver() as solver:
        states = [declare(i) for i in range(K)]
        solver.add_assertion(init(states[i], a, b))
        solver.add_assertion(Equals(states[-1]['pc'], Int(4)))
        for k in range(K):
            if k>0:
                solver.add_assertion(trans(states[k-1], states[k], a, b))
        if solver.solve():
            print(f"r: {solver.get_value(states[-1]['r'])}, s: {solver.get_value(states[-1]['s'])}, t: {solver.get_value(states[-1]['t'])}")
        else:
            print("> Not feasible.")
            
def bmc_always(inv, i, a, b, K):
    
    with Solver() as solver:
        states = [declare(i) for i in range(K)]
        solver.add_assertion(init(states[i], a, b))
        solver.add_assertion(Equals(states[-1]['pc'], Int(4)))
        for k in range(K):
            if k>0:
                solver.add_assertion(trans(states[k-1], states[k], a, b))
            solver.push()
            solver.add_assertion(Not(inv(states[k])))
            if solver.solve():
                print(f"> Invariant does not hold for {k+1} first states. Counter-example:")
                for i,state in enumerate(states[:k+1]):
                    print(f"> State {i}: pc = {solver.get_value(state['pc'])}\nq = {solver.get_value(state['q'])}\ns = {solver.get_value(state['s'])}\nt = {solver.get_value(state['t'])}\nr = {solver.get_value(state['r'])}\ns' = {solver.get_value(state['s_prox'])}\nt' = {solver.get_value(state['t_prox'])}\nr' = {solver.get_value(state['r_prox'])}")
                return
            else:
                if k==K-1:
                    print(f"> Invariant holds for the first {K} states.")
                else:
                    solver.pop()

def bezout(state):
    a = Int(a)
    b = Int(b)
    times_a = Times(a, state['s'])
    times_b = Times(b, state['t'])
    plus_a_b = Plus(times_a, times_b)
    
    return Equals(plus_a_b, state['r'])

In [ ]:
i = int(input('Insira o número do estado inicial: '))
K = int(input('Insira o número de estados: '))

In [ ]:
a = int(input('Insira o valor de a: '))
b = int(input('Insira o valor de b: '))

In [ ]:
gera_traco(i, a, b, K)
bmc_always(bezout, i, a, b, K)